# ST449 Final Project

## Connect4 Best Bots and Modifications

In [ ]:
# imports
import numpy as np
import sys
import pygame
import math
import random
import pandas as pd
from collections import namedtuple, defaultdict, deque
import time
from itertools import permutations
# from aima_python_master.utils4e  import *
# from aima_python_master.games4e  import *


### Generating the Board

#### Creating Connect Four Game class

In [ ]:
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))
                    

In [ ]:
class C4(Game):
    """A TicTacToe-like game in which you can only make a move on the bottom
    row, or in a square directly above an occupied square. Traditionally
    played on a 6*7 board and requiring 4 in a row."""

    def __init__(self, h=6, v=7, k=4):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """ If we write (x, y) as the coordinate on the board,
        then the bottom row correspond to x=7, or equivalently x=self.h
        Recall that state.board is a dict and the keys are occupied locations. """
        return [(x, y) for (x, y) in state.moves
                if x == self.h or (x + 1 , y) in state.board]

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k
        
        

#### Evaluation function

In [ ]:
def generate_segments(h=6, v=7, k=4):
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments = generate_segments()

def count_in_segment(segment, state):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count = 0, 0
    for x, y in segment:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        elif state.board.get((x, y)) == 'O':
            O_count += 1
    return X_count, O_count

def eval_segment(segment, state, player):
    """ Returns the evaluation score for a segment """
    X_count, O_count = count_in_segment(segment, state)
    if X_count > 0 and O_count > 0:
        return 0   # mixed segments are neutral

    count = max(X_count, O_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if X_count > O_count:
        dominant = 'X'
    else:
        dominant = 'O'

    if dominant == player:
        return score
    else:
        return -score

def eval_fn(state, player):
    """ The evaluation function """
    total = 0
    for segment in all_segments:
        total += eval_segment(segment, state, player)
    return total


### Search algorithms

#### Alpha-beta cutoff search

In [ ]:
def alpha_beta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

#### Monte Carlo tree search

In [ ]:
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}
        self.actions = None


def ucb(n, C=1.4):
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)

def monte_carlo_tree_search(state, game, N=20000):
    def select(n):
        """select a leaf node in the tree"""
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """expand the leaf node by adding all its children states"""
        if not n.children and not game.terminal_test(n.state):
            n.children = {MCT_Node(state=game.result(n.state, action), parent=n): action
                          for action in game.actions(n.state)}
        return select(n)

    def simulate(game, state):
        """simulate the utility of current state by random picking a step"""
        player = game.to_move(state)
        while not game.terminal_test(state):
            action = random.choice(list(game.actions(state)))
            state = game.result(state, action)
        v = game.utility(state, player)
        return -v

    def backprop(n, utility):
        """passing the utility back to all parent nodes"""
        if utility > 0:
            n.U += utility
        # if utility == 0:
        #     n.U += 0.5
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)

    for _ in range(N):
        leaf = select(root)
        child = expand(leaf)
        result = simulate(game, child.state)
        backprop(child, result)

    max_state = max(root.children, key=lambda p: p.N)

    return root.children.get(max_state)


#### Define the bots

In [ ]:
def standard_MC_bot(game, state):
    return monte_carlo_tree_search(state, game, N = 1000)

def standard_alpha_beta_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 5)

def standard_alpha_beta_eval_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 5, eval_fn = eval_fn)
    

### Play Game

#### Standard

In [ ]:
testC4game = C4()

In [ ]:
testC4game.play_game(standard_MC_bot, standard_alpha_beta_bot) # Returns 1 for player 1 winning (X), -1 for player 2 winning (O), 0 for tie

#### Hyperparameter Tuning

In [ ]:
def test_MC_bot_1000(game, state):
    return monte_carlo_tree_search(state, game, N = 1000)

def test_MC_bot_10000(game, state):
    return monte_carlo_tree_search(state, game, N = 10000)

def test_MC_bot_5000(game, state):
    return monte_carlo_tree_search(state, game, N = 5000)

def test_alpha_beta_bot_3(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3)

def test_alpha_beta_bot_4(game, state):
    return alpha_beta_cutoff_search(state, game, d = 4)

def test_alpha_beta_bot_5(game, state):
    return alpha_beta_cutoff_search(state, game, d = 5)

def test_alpha_beta_eval_bot_3(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3, eval_fn = eval_fn)

def test_alpha_beta_eval_bot_4(game, state):
    return alpha_beta_cutoff_search(state, game, d = 4, eval_fn = eval_fn)

def test_alpha_beta_eval_bot_5(game, state):
    return alpha_beta_cutoff_search(state, game, d = 5, eval_fn = eval_fn)


In [ ]:
def calculate_heuristic(wins, total_time):
    return wins / total_time if total_time > 0 else 0

# Simulation cache and results DataFrame
simulation_cache = {}
results_df = pd.DataFrame(columns=[
    "Bot1", "Bot2", "Bot1_Wins", "Bot2_Wins", "Bot1_Time", "Bot2_Time", "Bot1_Heuristic", "Bot2_Heuristic"
])

# Function to run simulation and calculate heuristic scores
def run_simulation_and_calculate_scores(bot1, bot2, num_games=30):
    bot_pair_key = (bot1[1], bot2[1])  # Use bot labels as cache key

    # Check if results are already cached
    if bot_pair_key in simulation_cache:
        return simulation_cache[bot_pair_key]

    results = {"bot1_wins": 0, "bot2_wins": 0, "bot1_time": 0, "bot2_time": 0}

    # Each bot alternates as the starting player for half the games
    for i in range(num_games):
        if i % 2 == 0:
            game_results = run_simulation(1, [(bot1[0], bot1[1]), (bot2[0], bot2[1])])
        else:
            game_results = run_simulation(1, [(bot2[0], bot2[1]), (bot1[0], bot1[1])])

        results["bot1_wins"] += game_results[bot1[1]]["wins"]
        results["bot2_wins"] += game_results[bot2[1]]["wins"]
        results["bot1_time"] += game_results[bot1[1]]["time_per_move"]
        results["bot2_time"] += game_results[bot2[1]]["time_per_move"]

    bot1_heuristic = calculate_heuristic(results["bot1_wins"], results["bot1_time"])
    bot2_heuristic = calculate_heuristic(results["bot2_wins"], results["bot2_time"])

    # Cache the results
    simulation_cache[bot_pair_key] = (bot1_heuristic, bot2_heuristic)

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [
        bot1[1], bot2[1],
        results["bot1_wins"], results["bot2_wins"],
        results["bot1_time"], results["bot2_time"],
        bot1_heuristic, bot2_heuristic
    ]
    
    return bot1_heuristic, bot2_heuristic

# Function to create heuristic matrix and format as DataFrame
def create_heuristic_matrix_df(bots, opponents):
    bot_names = [bot[1] for bot in bots]
    opponent_names = [opponent[1] for opponent in opponents]

    # Initialize matrix with zeros
    matrix = np.zeros((len(bots), len(opponents) + 1))

    for i, bot in enumerate(bots):
        total_heuristic = 0
        for j, opponent in enumerate(opponents):
            if bot != opponent:
                bot_heuristic, _ = run_simulation_and_calculate_scores(bot, opponent)
                matrix[i, j] = bot_heuristic
                total_heuristic += bot_heuristic

        # Calculate average heuristic score
        if len(opponents) > 0:
            matrix[i, -1] = total_heuristic / len(opponents)

    # Create DataFrame with column headers
    column_headers = opponent_names + ["Average"]
    heuristic_df = pd.DataFrame(matrix, columns=column_headers)
    heuristic_df.insert(0, "Bot", bot_names)  # Insert bot names as the first column

    return heuristic_df

# Function to run a simulation of games
def run_simulation(num_games, bot_functions_with_labels):
    results = {label: {"wins": 0, "total_time": 0, "total_moves": 0} for _, label in bot_functions_with_labels}
    bot_order = deque([bot for bot, _ in bot_functions_with_labels])
    label_order = deque([label for _, label in bot_functions_with_labels])

    for i in range(num_games):
        bot_order.rotate(-1)
        label_order.rotate(-1)
        X_bot, O_bot = bot_order
        X_label, O_label = label_order
        game = C4(h=6, v=7, k=4)
        state = game.initial
        current_player = 'X'
        game_over = False
        player_to_bot = {'X': X_bot, 'O': O_bot}
        player_to_label = {'X': X_label, 'O': O_label}

        while not game_over:
            bot = player_to_bot[current_player]
            start_time = time.time()
            move = bot(game, state)
            state = game.result(state, move)
            move_time = time.time() - start_time
            bot_label = player_to_label[current_player]
            results[bot_label]["total_time"] += move_time
            results[bot_label]["total_moves"] += 1

            if game.terminal_test(state):
                winner = game.utility(state, game.to_move(game.initial))
                if winner == 1:
                    results[X_label]["wins"] += 1
                elif winner == -1:
                    results[O_label]["wins"] += 1
                game_over = True

            current_player = 'O' if current_player == 'X' else 'X'

    for bot in results:
        if results[bot]["total_moves"] > 0:
            results[bot]["time_per_move"] = results[bot]["total_time"] / results[bot]["total_moves"]
        else:
            results[bot]["time_per_move"] = 0

    return results

# Define the bots
monte_carlo_bots = [
    (test_MC_bot_1000, "MC_1000"),
    (test_MC_bot_10000, "MC_10000"),
    (test_MC_bot_5000, "MC_5000")
]

alpha_beta_bots = [
    (test_alpha_beta_bot_4, "AB_4"),
    (test_alpha_beta_bot_5, "AB_5"),
    (test_alpha_beta_eval_bot_3, "AB_3")
]

alpha_beta_eval_bots = [
    (test_alpha_beta_eval_bot_4, "ABE_4"),
    (test_alpha_beta_eval_bot_5, "ABE_5"),
    (test_alpha_beta_eval_bot_3, "ABE_3")
]

# Create matrices for each type of bots
monte_carlo_df = create_heuristic_matrix_df(monte_carlo_bots, alpha_beta_bots + alpha_beta_eval_bots)
alpha_beta_df = create_heuristic_matrix_df(alpha_beta_bots, monte_carlo_bots + alpha_beta_eval_bots)
alpha_beta_eval_df = create_heuristic_matrix_df(alpha_beta_eval_bots, monte_carlo_bots + alpha_beta_bots)

# Display the results
print("\nMonte Carlo Bots Heuristic Matrix:")
print(monte_carlo_df)

print("\nAlpha Beta Bots Heuristic Matrix:")
print(alpha_beta_df)

print("\nAlpha Beta Eval Bots Heuristic Matrix:")
print(alpha_beta_eval_df)

In [ ]:
# from collections import defaultdict
# import numpy as np
# import time

# # Define the bots
# monte_carlo_bots = [
#     (test_MC_bot_1000, "MC_1000"),
#     (test_MC_bot_10000, "MC_10000"),
#     (test_MC_bot_5000, "MC_5000")
# ]

# alpha_beta_bots = [
#     (test_alpha_beta_bot_4, "AB_4"),
#     (test_alpha_beta_bot_5, "AB_5"),
#     (test_alpha_beta_eval_bot_3, "AB_3")
# ]

# alpha_beta_eval_bots = [
#     (test_alpha_beta_eval_bot_4, "ABE_4"),
#     (test_alpha_beta_eval_bot_5, "ABE_5"),
#     (test_alpha_beta_eval_bot_3, "ABE_3")
# ]


# # Cache for simulation results
# simulation_cache = {}

# # DataFrame to store match results
# columns = ["Bot1", "Bot2", "Bot1_Wins", "Bot2_Wins", "Bot1_Time", "Bot2_Time", "Bot1_Heuristic", "Bot2_Heuristic"]
# results_df = pd.DataFrame(columns=columns)

# # Function to calculate heuristic score
# def calculate_heuristic(wins, total_time):
#     return wins / total_time if total_time > 0 else 0

# # Function to run simulation and calculate heuristic scores
# def run_simulation_and_calculate_scores(bot1, bot2, num_games=30):
#     bot_pair_key = (bot1[1], bot2[1])  # Use bot labels as cache key

#     # Check if results are already cached
#     if bot_pair_key in simulation_cache:
#         return simulation_cache[bot_pair_key]
    
#     results = {"bot1_wins": 0, "bot2_wins": 0, "bot1_time": 0, "bot2_time": 0}

#     for i in range(num_games):
#         if i % 2 == 0:
#             game_results = run_simulation(1, [(bot1[0], bot1[1]), (bot2[0], bot2[1])])
#         else:
#             game_results = run_simulation(1, [(bot2[0], bot2[1]), (bot1[0], bot1[1])])

#         results["bot1_wins"] += game_results[bot1[1]]["wins"]
#         results["bot2_wins"] += game_results[bot2[1]]["wins"]
#         results["bot1_time"] += game_results[bot1[1]]["time_per_move"]
#         results["bot2_time"] += game_results[bot2[1]]["time_per_move"]

#     bot1_heuristic = calculate_heuristic(results["bot1_wins"], results["bot1_time"])
#     bot2_heuristic = calculate_heuristic(results["bot2_wins"], results["bot2_time"])

#     # Cache the results
#     simulation_cache[bot_pair_key] = (bot1_heuristic, bot2_heuristic)

#     # Append results to DataFrame
#     results_df.loc[len(results_df)] = [
#         bot1[1], bot2[1],
#         results["bot1_wins"], results["bot2_wins"],
#         results["bot1_time"], results["bot2_time"],
#         bot1_heuristic, bot2_heuristic
#     ]
    
#     return bot1_heuristic, bot2_heuristic

# # Function to create heuristic matrix
# def create_heuristic_matrix(bots, opponents):
#     num_bots = len(bots)
#     num_opponents = len(opponents)
#     matrix = np.zeros((num_bots, num_opponents + 1))  # Last column for averages

#     for i, bot in enumerate(bots):
#         total_heuristic = 0
#         for j, opponent in enumerate(opponents):
#             if bot != opponent:
#                 bot_heuristic, _ = run_simulation_and_calculate_scores(bot, opponent)
#                 matrix[i][j] = bot_heuristic
#                 total_heuristic += bot_heuristic

#         # Calculate average heuristic score
#         if num_opponents > 0:
#             matrix[i, -1] = total_heuristic / num_opponents
    
#     return matrix

# # Function to run a simulation of games
# def run_simulation(num_games, bot_functions_with_labels):
#     results = {label: {"wins": 0, "total_time": 0, "total_moves": 0} for _, label in bot_functions_with_labels}
#     bot_order = deque([bot for bot, _ in bot_functions_with_labels])
#     label_order = deque([label for _, label in bot_functions_with_labels])

#     for i in range(num_games):
#         bot_order.rotate(-1)
#         label_order.rotate(-1)
#         X_bot, O_bot = bot_order
#         X_label, O_label = label_order
#         game = C4(h=6, v=7, k=4)
#         state = game.initial
#         current_player = 'X'
#         game_over = False
#         player_to_bot = {'X': X_bot, 'O': O_bot}
#         player_to_label = {'X': X_label, 'O': O_label}

#         while not game_over:
#             bot = player_to_bot[current_player]
#             start_time = time.time()
#             move = bot(game, state)
#             state = game.result(state, move)
#             move_time = time.time() - start_time
#             bot_label = player_to_label[current_player]
#             results[bot_label]["total_time"] += move_time
#             results[bot_label]["total_moves"] += 1

#             if game.terminal_test(state):
#                 winner = game.utility(state, game.to_move(game.initial))
#                 if winner == 1:
#                     results[X_label]["wins"] += 1
#                 elif winner == -1:
#                     results[O_label]["wins"] += 1
#                 game_over = True

#             current_player = 'O' if current_player == 'X' else 'X'

#     for bot in results:
#         if results[bot]["total_moves"] > 0:
#             results[bot]["time_per_move"] = results[bot]["total_time"] / results[bot]["total_moves"]
#         else:
#             results[bot]["time_per_move"] = 0

#     return results

# # Create matrices for each type of bots
# monte_carlo_matrix = create_heuristic_matrix(monte_carlo_bots, alpha_beta_bots + alpha_beta_eval_bots)
# alpha_beta_matrix = create_heuristic_matrix(alpha_beta_bots, monte_carlo_bots + alpha_beta_eval_bots)
# alpha_beta_eval_matrix = create_heuristic_matrix(alpha_beta_eval_bots, monte_carlo_bots + alpha_beta_bots)

# # Print the matrices
# print("Monte Carlo Bots Heuristic Matrix:")
# print(monte_carlo_matrix)

# print("\nAlpha Beta Bots Heuristic Matrix:")
# print(alpha_beta_matrix)

# print("\nAlpha Beta Eval Bots Heuristic Matrix:")
# print(alpha_beta_eval_matrix)

# # Print the results DataFrame
# print("\nMatch Results DataFrame:")
# print(results_df)



In [ ]:
# display(results_df)
# results_df.to_csv('30_game_simulations.csv')

In [ ]:
# # Assuming df is your original DataFrame
# # Combine the results for each bot regardless of whether it was Bot1 or Bot2
# df_combined = pd.concat([
#     results_df.rename(columns={"Bot1": "Bot", "Bot2": "Opponent", "Bot1_Wins": "Wins_as_P1", "Bot2_Wins": "Wins_as_P2", "Bot1_Time": "Time_as_P1", "Bot2_Time": "Time_as_P2"}),
#     results_df.rename(columns={"Bot2": "Bot", "Bot1": "Opponent", "Bot2_Wins": "Wins_as_P1", "Bot1_Wins": "Wins_as_P2", "Bot2_Time": "Time_as_P1", "Bot1_Time": "Time_as_P2"})
# ])



# # Group by Bot and calculate the sum of wins and total time
# grouped_df = df_combined.groupby("Bot").agg({"Wins_as_P1": "sum", "Time_as_P1": "sum"}).reset_index()

# # Calculate the heuristic eval metric and average time for each bot
# grouped_df["Heuristic_Eval_Metric"] = (grouped_df["Wins_as_P1"] / 360) / (grouped_df["Time_as_P1"] / 12)
# # grouped_df["Average_Time"] = 

# # Save the DataFrame locally as a CSV file
# grouped_df.to_csv('heuristic_eval_metrics.csv', index=False)

# print("DataFrame saved locally as 'heuristic_eval_metrics.csv'.")

In [ ]:
# grouped_df

#### Different sized boards

In [ ]:
# Create utility function used for evaluation in the alpha_beta_eval_bot to account for different sized boards [4, 4]
h_1 = 4
v_1 = 4

def generate_segments_diff_size_4x4(h = h_1, v = v_1, k = 4):  
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments_diff_size_4x4 = generate_segments_diff_size_4x4()

def count_in_segment_diff_size_4x4(segment, state):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count = 0, 0
    for x, y in segment:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        elif state.board.get((x, y)) == 'O':
            O_count += 1
    return X_count, O_count

def eval_segment_diff_size_4x4(segment, state, player):
    """ Returns the evaluation score for a segment """
    X_count, O_count = count_in_segment_diff_size_4x4(segment, state)
    if X_count > 0 and O_count > 0:
        return 0   # mixed segments are neutral

    count = max(X_count, O_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if X_count > O_count:
        dominant = 'X'
    else:
        dominant = 'O'

    if dominant == player:
        return score
    else:
        return -score

def eval_fn_4x4(state, player):
    """ The evaluation function """
    total = 0
    for segment in all_segments_diff_size_4x4:
        total += eval_segment_diff_size_4x4(segment, state, player)
    return total

testC4game_4x4 = C4(h = h_1, v = v_1)


In [ ]:
# Create utility function used for evaluation in the alpha_beta_eval_bot to account for different sized boards [2, 15]
h_2 = 2
v_2 = 15

def generate_segments_diff_size_2x15(h = h_2, v = v_2, k = 4):  
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments_diff_size_2x15 = generate_segments_diff_size_2x15()

def count_in_segment_diff_size_2x15(segment, state):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count = 0, 0
    for x, y in segment:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        elif state.board.get((x, y)) == 'O':
            O_count += 1
    return X_count, O_count

def eval_segment_diff_size_2x15(segment, state, player):
    """ Returns the evaluation score for a segment """
    X_count, O_count = count_in_segment_diff_size_2x15(segment, state)
    if X_count > 0 and O_count > 0:
        return 0   # mixed segments are neutral

    count = max(X_count, O_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if X_count > O_count:
        dominant = 'X'
    else:
        dominant = 'O'

    if dominant == player:
        return score
    else:
        return -score

def eval_fn_2x15(state, player):
    """ The evaluation function """
    total = 0
    for segment in all_segments_diff_size_2x15:
        total += eval_segment_diff_size_2x15(segment, state, player)
    return total

testC4game_2x15 = C4(h = h_2, v = v_2)


In [ ]:
# Create utility function used for evaluation in the alpha_beta_eval_bot to account for different sized boards [10, 10]
h_3 = 10
v_3 = 10

def generate_segments_diff_size_10x10(h = h_3, v = v_3, k = 4):  
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments_diff_size_10x10 = generate_segments_diff_size_10x10()

def count_in_segment_diff_size_10x10(segment, state):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count = 0, 0
    for x, y in segment:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        elif state.board.get((x, y)) == 'O':
            O_count += 1
    return X_count, O_count

def eval_segment_diff_size_10x10(segment, state, player):
    """ Returns the evaluation score for a segment """
    X_count, O_count = count_in_segment_diff_size_10x10(segment, state)
    if X_count > 0 and O_count > 0:
        return 0   # mixed segments are neutral

    count = max(X_count, O_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if X_count > O_count:
        dominant = 'X'
    else:
        dominant = 'O'

    if dominant == player:
        return score
    else:
        return -score

def eval_fn_10x10(state, player):
    """ The evaluation function """
    total = 0
    for segment in all_segments_diff_size_10x10:
        total += eval_segment_diff_size_10x10(segment, state, player)
    return total

testC4game_10x10 = C4(h = h_3, v = v_3)


In [ ]:
def diff_size_MC_bot(game, state):
    return monte_carlo_tree_search(state, game, N = 1000)

def diff_size_alpha_beta_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 4)

def diff_size_4x4_alpha_beta_eval_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3, eval_fn = eval_fn_4x4)

def diff_size_2x15_alpha_beta_eval_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3, eval_fn = eval_fn_2x15)

def diff_size_10x10_alpha_beta_eval_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3, eval_fn = eval_fn_10x10)
    

In [ ]:
random.seed(123)

def diff_size_run_simulation(num_games, bot_functions_with_labels, diff_size_game):
    """
    Run a simulation of games, alternating which bot goes first, second, and third.
    The bots are passed along with their custom labels.
    
    Parameters:
    - num_games: The number of games to simulate.
    - bot_functions_with_labels: A list of tuples where each tuple contains a bot function and its custom label.
    
    Returns:
    - results: A dictionary with the bot labels as keys, storing the number of wins and average time per move.
    """
    # Initialize results structure for each bot
    results = {
        label: {
            "overall": {"wins": 0, "total_time": 0, "total_moves": 0},
            "head_to_head": {opponent: {"first": {"wins": 0}, "second": {"wins": 0}} 
                             for _, opponent in bot_functions_with_labels if opponent != label}
        }
        for _, label in bot_functions_with_labels
    }
    results["board_size"] = f"{diff_size_game.h}x{diff_size_game.v} Board Size"

    # Generate all permutations of bot positions
    bot_permutations = list(permutations(bot_functions_with_labels, 2))
    num_permutations = len(bot_permutations)
    
    # Ensure an equal number of all permutations
    games_per_permutation = num_games // num_permutations

    for i in range(num_games):
        # Determine the current permutation to use
        current_permutation = bot_permutations[i % num_permutations]

        # Assign bots to positions (X, O, 3)
        X_bot, O_bot = [bot for bot, _ in current_permutation]
        X_label, O_label = [label for _, label in current_permutation]

        # Print the player assignments for this game
        print(f"Game {i+1}: X = {X_label}, O = {O_label}")

        # Now we play the game with X_bot, O_bot, T_bot as players
        game = diff_size_game  # The Connect4 board with different sizes
        state = game.initial
        
        current_player = 'X'
        game_over = False
        player_to_bot = {'X': X_bot, 'O': O_bot}
        player_to_label = {'X': X_label, 'O': O_label}

        while not game_over:
            # Get the bot for the current player and measure time for the move
            bot = player_to_bot[current_player]
            start_time = time.time()
            
            # Bot makes a move
            move = bot(game, state)  # Assuming this function returns the best move
            state = game.result(state, move)  # Apply the move and get the new game state
            
            move_time = time.time() - start_time
            bot_label = player_to_label[current_player]  # Get the label for the current bot
            results[bot_label]["overall"]["total_time"] += move_time  # Add time to the corresponding bot
            results[bot_label]["overall"]["total_moves"] += 1  # Increment the move count

            # Check if the game is over
            if game.terminal_test(state):
                winner = game.utility(state, game.to_move(game.initial))
                if winner == 1:
                    results[X_label]["overall"]["wins"] += 1
                    results[X_label]["head_to_head"][O_label]["first"]["wins"] += 1
                elif winner == -1:
                    results[O_label]["overall"]["wins"] += 1
                    results[O_label]["head_to_head"][X_label]["second"]["wins"] += 1
                game_over = True

            # Rotate players: X -> O -> X
            current_player = game.to_move(state)

        # Print the final board state
        print("Final board state:")
        game.display(state)
        print("-" * 40)

    # After all games, calculate average times per move
    for bot in results:
        if bot == "board_size":
            continue
        elif results[bot]["overall"]["total_moves"] > 0:
            results[bot]["overall"]["time_per_move"] = results[bot]["overall"]["total_time"] / results[bot]["overall"]["total_moves"]
        else:
            results[bot]["overall"]["time_per_move"] = 0

    return results

# Function to display results
def display_results(results):
    print(f"Simulation Results on {results["board_size"]}:")
    for bot, data in results.items():
        if bot == "board_size":
            continue
        print(f"Bot {bot}:")
        print(f"  Wins: {data["overall"]['wins']}")
        print(f"  Average Time per Move: {data["overall"]['time_per_move']:.4f} seconds")
        print("-" * 40)
        for opponent, matchups in data["head_to_head"].items():
            print(f"    Vs {opponent}:")
            print(f"      Wins as First Player: {matchups['first']['wins']}")
            print(f"      Wins as Second Player: {matchups['second']['wins']}")
        print("-" * 40)

num_games = 90

sizes = ["4x4", "2x15", "10x10"]
for size in sizes:
    game_name = f"testC4game_{size}"
    alpha_beta_eval_bot_name = f"diff_size_{size}_alpha_beta_eval_bot"

    game = globals()[game_name]
    alpha_beta_eval_bot = globals()[alpha_beta_eval_bot_name]

    bot_functions_with_labels = [
        (diff_size_MC_bot, "MCT"),
        (diff_size_alpha_beta_bot, "AlphaBeta"),
        (alpha_beta_eval_bot, "AlphaBetaEval")
    ]
    
    results = diff_size_run_simulation(num_games, bot_functions_with_labels, diff_size_game = game)

    # Display the results
    display_results(results)


#### 3-players

In [ ]:
class C4_3_player(Game):
    """
    A TicTacToe-like game in which you can only make a move on the bottom
    row, or in a square directly above an occupied square. This game introduces a third player that will play, the players take turns sequentially X,O,3
    """
    def __init__(self, h=6, v=7, k=4):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        # """Legal moves are any square not yet taken."""
        """ If we write (x, y) as the coordinate on the board,
        then the bottom row correspond to x=7, or equivalently x=self.h
        Recall that state.board is a dict and the keys are occupied locations. """
        # return state.moves
        return [(x, y) for (x, y) in state.moves
                if x == self.h or (x + 1 , y) in state.board]

    def result(self, state, move):
        """Apply a move and return the new state."""
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)

        # Determines the next player, in our case we have 3 players so follows a pattern 
        next_player = self.get_next_player(state.to_move)

        return GameState(to_move=next_player,
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the utility value for the given player."""
        if state.utility == 1:  # Player 1 (X) wins
            return 1 if player == 'X' else -1
        elif state.utility == -1:  # Player 2 (O) wins
            return 1 if player == 'O' else -1
        elif state.utility == 2:  # Player 3 (3) wins
            return 2 if player == '3' else -2
        return 0  # No winner yet
    

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If a player wins with this move, return a specific utility."""
        if (self.k_in_row(board, move, player, (0, 1)) or  # Horizontal
                self.k_in_row(board, move, player, (1, 0)) or  # Vertical
                self.k_in_row(board, move, player, (1, -1)) or  # Diagonal /
                self.k_in_row(board, move, player, (1, 1))):  # Diagonal \
            if player == 'X':
                return 1  # X wins
            elif player == 'O':
                return -1  # O wins
            elif player == '3':
                return 2  # Player 3 wins
        return 0  # No winner


    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k
    
    def get_next_player(self, current_player):
        """Cycle through the three players: X -> O -> 3 -> X."""
        return {'X': 'O', 'O': '3', '3': 'X'}[current_player]

#### Eval Function For 3 Player

The eval function for the AlphaBeta bot with the eval function now has to change. As it needs to account for segments for three different players.

In [ ]:
def generate_segments_3_player(h=6, v=7, k=4):
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments_three = generate_segments_3_player()

def count_in_segment_three(segment, state):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count, three_count = 0, 0, 0
    for x, y in segment:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        elif state.board.get((x, y)) == 'O':
            O_count += 1
        elif state.board.get((x, y)) == '3': # include the counts for the third player
            three_count += 1
    return X_count, O_count, three_count

def eval_segment_three(segment, state, player):
    """ Returns the evaluation score for a segment """
    X_count, O_count, three_count = count_in_segment_three(segment, state)
    if (X_count > 0 and O_count > 0) or (X_count > 0 and three_count >0) or (O_count > 0 and three_count > 0):
        return 0   # mixed segments are neutral

    count = max(X_count, O_count, three_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if (X_count > O_count) and (X_count > three_count):
        dominant = 'X'
    elif (O_count > three_count) and (O_count > X_count):
        dominant = 'O'
    else:
        dominant = '3' # addition of third player
        
    if dominant == player:
        return score
    else:
        return -score

def eval_fn_three(state, player):
    """ The evaluation function """
    total = 0
    for segment in all_segments_three:
        total += eval_segment_three(segment, state, player)
    return total


#### Bots for 3 player Game

In [ ]:
def test_MC_bot_three(game, state):
    return monte_carlo_tree_search(state, game, N = 1000)

def test_alpha_beta_bot_three(game, state):
    return alpha_beta_cutoff_search(state, game, d = 4)

def test_alpha_beta_eval_bot_three(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3, eval_fn = eval_fn_three)

def random_bot_three(game, state):
    """A bot that chooses a random legal move."""
    return random.choice(game.actions(state))

    

In [ ]:
testC4game_3 = C4_3_player()
testC4game_3.play_game(test_MC_bot_three, test_alpha_beta_eval_bot_three, random_bot_three)

In [ ]:
random.seed(123)

def run_simulation(num_games, bot_functions_with_labels):
    """
    Run a simulation of games, alternating which bot goes first, second, and third.
    The bots are passed along with their custom labels.
    
    Parameters:
    - num_games: The number of games to simulate.
    - bot_functions_with_labels: A list of tuples where each tuple contains a bot function and its custom label.
    
    Returns:
    - results: A dictionary with the bot labels as keys, storing the number of wins and average time per move.
    """
    # Initialize results structure for each bot
    results = {label: {
        "overall" :{"wins": 0, "total_time": 0, "total_moves": 0},
        "position" : {"first":{"wins": 0}, "second": {"wins":0}, "third":{"wins": 0}}}
        for _, label in bot_functions_with_labels}

    position_wins = {'X': 0, 'O': 0, '3': 0}

    # Generate all permutations of bot positions
    bot_permutations = list(permutations(bot_functions_with_labels))
    num_permutations = len(bot_permutations)
    
    # Ensure an equal number of all permutations
    games_per_permutation = num_games // num_permutations

    for i in range(num_games):
        # Determine the current permutation to use
        current_permutation = bot_permutations[i % num_permutations]

        # Assign bots to positions (X, O, 3)
        X_bot, O_bot, T_bot = [bot for bot, _ in current_permutation]
        X_label, O_label, T_label = [label for _, label in current_permutation]

        # Print the player assignments for this game
        print(f"Game {i+1}: X = {X_label}, O = {O_label}, 3 = {T_label}")

        # Now we play the game with X_bot, O_bot, T_bot as players
        game = C4_3_player(h=6, v=7, k=4)  # Standard Connect 4 board with 6x7 grid
        state = game.initial
        
        current_player = 'X'
        game_over = False
        player_to_bot = {'X': X_bot, 'O': O_bot, '3': T_bot}
        player_to_label = {'X': X_label, 'O': O_label, '3': T_label}

        while not game_over:
            # Get the bot for the current player and measure time for the move
            bot = player_to_bot[current_player]
            start_time = time.time()
            
            # Bot makes a move
            move = bot(game, state)  # Assuming this function returns the best move
            state = game.result(state, move)  # Apply the move and get the new game state
            
            move_time = time.time() - start_time
            bot_label = player_to_label[current_player]  # Get the label for the current bot
            results[bot_label]["overall"]["total_time"] += move_time  # Add time to the corresponding bot
            results[bot_label]["overall"]["total_moves"] += 1  # Increment the move count

            # Check if the game is over
            if game.terminal_test(state):
                winner = game.utility(state, game.to_move(game.initial))
                if winner == 1:
                    results[X_label]["overall"]["wins"] += 1
                    position_wins['X'] += 1
                    results[X_label]["position"]["first"]["wins"] +=1
                    print(f"Winner: {X_label}")
                elif winner == -1:
                    results[O_label]["overall"]["wins"] += 1
                    position_wins['O'] += 1
                    results[O_label]["position"]["second"]["wins"] +=1
                    print(f"Winner: {O_label}")
                elif winner == -2:
                    results[T_label]["overall"]["wins"] += 1
                    position_wins['3'] += 1
                    results[T_label]["position"]["third"]["wins"] +=1
                    print(f"Winner: {T_label}")

                game_over = True

            # Rotate players: X -> O -> 3 -> X
            current_player = game.get_next_player(current_player)

        # Print the final board state
        print("Final board state:")
        game.display(state)
        print("-" * 40)
        
    # After all games, calculate average times per move
    for bot in results:
        if results[bot]["overall"]["total_moves"] > 0:
            results[bot]["overall"]["time_per_move"] = results[bot]["overall"]["total_time"] / results[bot]["overall"]["total_moves"]
        else:
            results[bot]["overall"]["time_per_move"] = 0

    return results, position_wins

# Function to display results
def display_results(results, position_wins):
    print("Simulation Results:")
    for bot, data in results.items():
        print(f"Bot {bot}:")
        print(f"  Wins: {data["overall"]['wins']}")
        print(f"     Wins as first player: {data["position"]["first"]["wins"]}")
        print(f"     Wins as second player: {data["position"]["second"]["wins"]}")
        print(f"     Wins as third player: {data["position"]["third"]["wins"]}")
        print(f"  Average Time per Move: {data["overall"]['time_per_move']:.4f} seconds")
        print("-" * 40)
    
    print("Wins by Player Position:")
    for position, wins in position_wins.items():
        print(f"  {position}: {wins} wins")
    print("-" * 40)

# Running the simulation for n games with any bots
bot_functions_with_labels = [
    (test_MC_bot_three, "MCT"),
    (test_alpha_beta_bot_three, "AlphaBeta"),
    (test_alpha_beta_eval_bot_three, "AlphaBetaEval")
]

num_games = 60
results, position_wins = run_simulation(num_games, bot_functions_with_labels)

# Display the results
display_results(results, position_wins)

#### Random "blocks"

In [ ]:
class C4_obstacles(Game):
    """A TicTacToe-like game in which you can only make a move on the bottom
    row, or in a square directly above an occupied square. Traditionally
    played on a 6*7 board and requiring 4 in a row."""

    def __init__(self, h=6, v=7, k=4, obstacles=None):
        self.h = h
        self.v = v
        self.k = k
        self.obstacles = obstacles or [] # List of obstacle positions (x, y)
        moves = [(x, y) for x in range(1, h + 1) for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """ If we write (x, y) as the coordinate on the board,
        then the bottom row correspond to x=7, or equivalently x=self.h
        Recall that state.board is a dict and the keys are occupied locations. """
        valid_moves = []
        for (x, y) in state.moves:
            if x == self.h and (x, y) not in self.obstacles:
                    valid_moves.append((x, y))
            elif (x + 1, y) in state.board or (x + 1, y) in self.obstacles:
                valid_move = True
                for row in range(x + 1, self.h + 1):
                    if (row, y) not in state.board and (row, y) not in self.obstacles:
                        valid_move = False
                        break
                if valid_move and (x, y) not in self.obstacles:
                    valid_moves.append((x, y))
        return valid_moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or not any(self.actions(state))

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                if (x, y) in self.obstacles:
                    print('#', end=' ') # Display the obstacles as '#'s
                else:
                    print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return + 1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while (x, y) in board and board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while (x, y) in board and board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    print(state.board)
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))
        

In [ ]:
# Dynamically update the utility function used for evaluation in the alpha_beta_eval_bot to account for the "obstacles"
def generate_segments_obstacles(h=6, v=7, k=4):
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments_obstacles = generate_segments_obstacles()

def count_in_segment_obstacles(segment, state, game):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count, obstacle_count = 0, 0, 0
    for x, y in segment:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        elif state.board.get((x, y)) == 'O':
            O_count += 1
        elif (x, y) in game.obstacles:
            obstacle_count += 1
    return X_count, O_count, obstacle_count

def eval_segment_obstacles(segment, state, player, game):
    """ Returns the evaluation score for a segment """
    X_count, O_count, obstacle_count = count_in_segment_obstacles(segment, state, game)

    if obstacle_count > 0:
        return 0
    
    if X_count > 0 and O_count > 0:
        return 0   # mixed segments are neutral

    count = max(X_count, O_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if X_count > O_count:
        dominant = 'X'
    else:
        dominant = 'O'

    if dominant == player:
        return score
    else:
        return -score

def eval_fn_obstacles(state, player, game):
    """ The evaluation function """
    total = 0
    for segment in all_segments_obstacles:
        total += eval_segment_obstacles(segment, state, player, game)
    return total


In [ ]:
def obstacles_MC_bot(game, state):
    return monte_carlo_tree_search(state, game, N = 1000)

def obstacles_alpha_beta_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 4)

def obstacles_alpha_beta_eval_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 3, eval_fn = lambda state, player: eval_fn_obstacles(state, player, game))


In [ ]:
def generate_random_obstacles(h, v, n):
    all_positions = [(x, y) for x in range(1, h + 1) for y in range(1, v + 1)]
    return random.sample(all_positions, n)

def obstacle_run_simulation(num_games, bot_functions_with_labels):
    """
    Run a simulation of games, alternating which bot goes first, second, and third.
    The bots are passed along with their custom labels.
    
    Parameters:
    - num_games: The number of games to simulate.
    - bot_functions_with_labels: A list of tuples where each tuple contains a bot function and its custom label.
    
    Returns:
    - results: A dictionary with the bot labels as keys, storing the number of wins and average time per move.
    """
    # Initialize results structure for each bot
    results = {
        label: {
            "overall": {"wins": 0, "total_time": 0, "total_moves": 0},
            "head_to_head": {opponent: {"first": {"wins": 0}, "second": {"wins": 0}} 
                             for _, opponent in bot_functions_with_labels if opponent != label}
        }
        for _, label in bot_functions_with_labels
    }

    # Generate all permutations of bot positions
    bot_permutations = list(permutations(bot_functions_with_labels, 2))
    num_permutations = len(bot_permutations)
    
    # Ensure an equal number of all permutations
    games_per_permutation = num_games // num_permutations

    seed = 123
    
    for i in range(num_games):
        
        if i % 6 == 0:
            random.seed(seed)
            seed += 1
            
            rand_obstacles = generate_random_obstacles(h = 6, v = 7, n = 8)
            game = C4_obstacles(obstacles = rand_obstacles) # The Connect4 board with 8 (~20% of board) randomly generated obstacles
        
        # Determine the current permutation to use
        current_permutation = bot_permutations[i % num_permutations]

        # Assign bots to positions (X, O, 3)
        X_bot, O_bot = [bot for bot, _ in current_permutation]
        X_label, O_label = [label for _, label in current_permutation]

        # Print the player assignments for this game
        print(f"Game {i+1}: X = {X_label}, O = {O_label}")

        # Now we play the game with X_bot, O_bot as players
        state = game.initial
        
        current_player = 'X'
        game_over = False
        player_to_bot = {'X': X_bot, 'O': O_bot}
        player_to_label = {'X': X_label, 'O': O_label}

        while not game_over:
            # Get the bot for the current player and measure time for the move
            bot = player_to_bot[current_player]
            start_time = time.time()
            
            # Bot makes a move
            move = bot(game, state)  # Assuming this function returns the best move
            state = game.result(state, move)  # Apply the move and get the new game state
            
            move_time = time.time() - start_time
            bot_label = player_to_label[current_player]  # Get the label for the current bot
            results[bot_label]["overall"]["total_time"] += move_time  # Add time to the corresponding bot
            results[bot_label]["overall"]["total_moves"] += 1  # Increment the move count

            # Check if the game is over
            if game.terminal_test(state):
                winner = game.utility(state, game.to_move(game.initial))
                if winner == 1:
                    results[X_label]["overall"]["wins"] += 1
                    results[X_label]["head_to_head"][O_label]["first"]["wins"] += 1
                elif winner == -1:
                    results[O_label]["overall"]["wins"] += 1
                    results[O_label]["head_to_head"][X_label]["second"]["wins"] += 1
                game_over = True

            # Rotate players: X -> O -> X
            current_player = game.to_move(state)

        # Print the final board state
        print("Final board state:")
        game.display(state)
        print("-" * 40)

    # After all games, calculate average times per move
    for bot in results:
        if results[bot]["overall"]["total_moves"] > 0:
            results[bot]["overall"]["time_per_move"] = results[bot]["overall"]["total_time"] / results[bot]["overall"]["total_moves"]
        else:
            results[bot]["overall"]["time_per_move"] = 0

    return results

# Function to display results
def display_results(results):
    print("Simulation Results:")
    for bot, data in results.items():
        print(f"Bot {bot}:")
        print(f"  Wins: {data["overall"]['wins']}")
        print(f"  Average Time per Move: {data["overall"]['time_per_move']:.4f} seconds")
        print("-" * 40)
        for opponent, matchups in data["head_to_head"].items():
            print(f"    Vs {opponent}:")
            print(f"      Wins as First Player: {matchups['first']['wins']}")
            print(f"      Wins as Second Player: {matchups['second']['wins']}")
        print("-" * 40)

num_games = 90

bot_functions_with_labels = [
    (obstacles_MC_bot, "MCT"),
    (obstacles_alpha_beta_bot, "AlphaBeta"),
    (obstacles_alpha_beta_eval_bot, "AlphaBetaEval")
]

game = C4_obstacles(obstacles = obstacle_list)
    
results = obstacle_run_simulation(num_games, bot_functions_with_labels)

# Display the results
display_results(results)
